## Gossipcop 

In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
from urllib.parse import urlparse # To parse and check URL scheme

In [20]:
# Load gossip fake
gossip_fake = pd.read_csv("gossipcop_fake.csv")
gossip_fake.head()

,id,news_url,title,tweet_ids
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...


In [21]:
# Inspect column attributes
gossip_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5323 entries, 0 to 5322
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         5323 non-null   object
 1   news_url   5067 non-null   object
 2   title      5323 non-null   object
 3   tweet_ids  5135 non-null   object
dtypes: object(4)
memory usage: 166.5+ KB


In [22]:
# Confirm dropped 
gossip_fake.shape

(5323, 4)

- Scrapping to extract text

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re # For cleaning up text
from urllib.parse import urlparse # Correctly imported now

# --- Configuration ---
# Set a User-Agent to mimic a web browser. Some sites block requests without this.
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
# Small delay between requests to avoid overwhelming servers and getting blocked.
REQUEST_DELAY_SECONDS = 0.5

# --- Load your gossip_fake DataFrame ---
# As per your image, assuming you load it from a CSV:
try:
    gossip_fake = pd.read_csv('gossipcop_fake.csv')
    print("DataFrame loaded from 'gossipcop_fake.csv'.")
except FileNotFoundError:
    print("Error: 'gossipcop_fake.csv' not found. Creating a sample DataFrame for demonstration.")
    # Sample DataFrame for demonstration if CSV is not found
    data = {
        'id': ['gossipcop-249749932', 'gossipcop-45800247171', 'gossipcop-941085037', 'gossipcop-2547091536', 'gossipcop-5470651226'],
        'news_url': [
            'www.dailymail.co.uk/tvshowbiz/article-5874213/Miley-Cyrus-Liam-Hemsworth-secretly-married.html',
            'hollywoodlife.com/2018/05/05/paris-jackson-cara-delevingne-enjoy-night-out-in-hollywood/',
            'variety.com/2017/biz/news/tax-march-donald-trump-celebrities-1202030282/',
            'www.dailymail.co.uk/femail/article-3499192/Cindy-Crawford-daughter-Kaia-Gerber-wears-amazing-new-campaign.html',
            'variety.com/2018/film/news/list-2018-oscars-nominations-1202673238/'
        ],
        'title': [
            'Did Miley Cyrus and Liam Hemsworth secretly ge...',
            'Paris Jackson & Cara Delevingne Enjoy Night Ou...',
            'Celebrities Join Tax March in Protest of Donal...',
            'Cindy Crawford\'s daughter Kaia Gerber wears a ...',
            'Full List of 2018 Oscar Nominations - Variety'
        ],
        'tweet_ids': [
            '284329075902926848/128432744559882361/12843935..',
            '9928985300267130880/992897935418520169/992899...',
            '853356935532028961/853359676548920128/853359..',
            '980821908151080801/980824208561722080/980825..',
            '95379270993242131/953795069925301249/953798..'
        ]
    }
    gossip_fake = pd.DataFrame(data)

# Ensure news_url column is clean (no NaNs)
# This step is crucial before attempting to scrape
initial_rows = len(gossip_fake)
gossip_fake = gossip_fake.dropna(subset=['news_url']).reset_index(drop=True)
if len(gossip_fake) < initial_rows:
    print(f"Dropped {initial_rows - len(gossip_fake)} rows with missing 'news_url'.")


# --- Function to scrape article text from a single URL ---
def scrape_article_text(url):
    """
    Fetches the content of a URL and extracts the main article text.
    Automatically prepends 'https://' if the URL has no scheme.

    Args:
        url (str): The URL of the web page to scrape.

    Returns:
        str: The extracted article text, or an informative error message if scraping fails.
    """
    # Standardize URL: Add 'https://' if no scheme is present
    parsed_url = urlparse(url)
    if not parsed_url.scheme:
        # Default to https, as most modern sites use it
        standardized_url = f"https://{url}"
    else:
        standardized_url = url

    try:
        # Send a GET request to the standardized URL
        response = requests.get(standardized_url, headers=HEADERS, timeout=15) # Increased timeout
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # --- Article Text Extraction Logic ---
        # This is a general approach. For more precise extraction, you might need
        # to inspect the HTML structure of the specific websites you are scraping
        # (e.g., using browser developer tools) and target specific tags/classes.
        # Common patterns include:
        # - <article> tag: soup.find('article')
        # - div with specific class: soup.find('div', class_='article-body')
        # - div with specific ID: soup.find('div', id='main-content')
        # For now, we'll collect text from all <p> tags, which is often effective.

        paragraphs = soup.find_all('p')
        text_content = ' '.join([p.get_text() for p in paragraphs])

        # Basic text cleaning: remove extra whitespace and newlines
        text_content = re.sub(r'\s+', ' ', text_content).strip()

        if not text_content:
            # If no paragraph text is found, try to get text from the body or a common article div
            body_text = soup.body.get_text(separator=' ', strip=True) if soup.body else ""
            if body_text and len(body_text) > 100: # Check if body text is substantial
                text_content = re.sub(r'\s+', ' ', body_text).strip()
            else:
                return f"[No substantial text found or page structure is unusual for {standardized_url}]"

        return text_content

    except requests.exceptions.MissingSchema:
        return f"[Error fetching URL: Invalid URL '{url}': Missing scheme (e.g., http:// or https://). This error should ideally be caught by standardization logic.]"
    except requests.exceptions.ConnectionError as e:
        return f"[Error fetching URL: Connection error for {standardized_url}: {e}]"
    except requests.exceptions.Timeout:
        return f"[Error fetching URL: Request timed out for {standardized_url}]"
    except requests.exceptions.HTTPError as e:
        return f"[Error fetching URL: HTTP error {e.response.status_code} for {standardized_url}]"
    except requests.exceptions.RequestException as e:
        return f"[Error fetching URL: General request error for {standardized_url}: {e}]"
    except Exception as e:
        return f"[An unexpected error occurred for {standardized_url}: {e}]"

# --- Apply the scraping function to the 'news_url' column ---
print("\nStarting article text extraction from URLs. This may take some time...")
print(f"Processing {len(gossip_fake)} entries.")

# Create a new column to store the extracted text
# Initialize with None or an empty string, depending on preference
gossip_fake['extracted_article_text'] = None

for index, row in gossip_fake.iterrows():
    url = row['news_url']
    if pd.notna(url): # Ensure URL is not NaN
        # print(f"Scraping {index+1}/{len(gossip_fake)}: {url}")
        extracted_text = scrape_article_text(url)
        gossip_fake.at[index, 'extracted_article_text'] = extracted_text
    else:
        gossip_fake.at[index, 'extracted_article_text'] = "[No URL provided]"

    time.sleep(REQUEST_DELAY_SECONDS) # Pause for politeness

print("\nArticle text extraction complete!")

# Display the DataFrame with the new 'extracted_article_text' column
print("\nDataFrame with Extracted Article Text (first 5 rows):")
print(gossip_fake[['title', 'news_url', 'extracted_article_text']].head())

# You can also save the DataFrame to a CSV or other format
gossip_fake.to_csv('gossip_fake_with_article_text.csv', index=False)


In [ ]:
gossiped_fake = pd.read_csv('gossip_fake_with_article_text.csv')
gossiped_fake.head()

In [ ]:
gossiped_fake.info()

### gossipcop_real.csv

In [37]:
# Load gossip real
gossip_real = pd.read_csv("gossipcop_real.csv")
gossip_real.head()

,id,news_url,title,tweet_ids
0,gossipcop-882573,https://www.brides.com/story/teen-mom-jenelle-...,Teen Mom Star Jenelle Evans' Wedding Dress Is ...,912371411146149888\t912371528343408641\t912372...
1,gossipcop-875924,https://www.dailymail.co.uk/tvshowbiz/article-...,Kylie Jenner refusing to discuss Tyga on Life ...,901989917546426369\t901989992074969089\t901990...
2,gossipcop-894416,https://en.wikipedia.org/wiki/Quinn_Perkins,Quinn Perkins,931263637246881792\t931265332022579201\t931265...
3,gossipcop-857248,https://www.refinery29.com/en-us/2018/03/19192...,I Tried Kim Kardashian's Butt Workout & Am For...,868114761723936769\t868122567910936576\t868128...
4,gossipcop-884684,https://www.cnn.com/2017/10/04/entertainment/c...,Celine Dion donates concert proceeds to Vegas ...,915528047004209152\t915529285171122176\t915530...


In [38]:
# Inspect column attributes
gossip_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16817 entries, 0 to 16816
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         16817 non-null  object
 1   news_url   16804 non-null  object
 2   title      16817 non-null  object
 3   tweet_ids  15759 non-null  object
dtypes: object(4)
memory usage: 525.7+ KB


In [41]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re # For cleaning up text
from urllib.parse import urlparse # Correctly imported now

# --- Configuration ---
# Set a User-Agent to mimic a web browser. Some sites block requests without this.
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
# Small delay between requests to avoid overwhelming servers and getting blocked.
REQUEST_DELAY_SECONDS = 0.5

# --- Load your gossip_real DataFrame ---
# As per your confirmation, gossip_real is loaded from 'gossipcop_real.csv'.
gossip_real = pd.read_csv('gossipcop_real.csv')
print("DataFrame loaded from 'gossipcop_real.csv'.")


# Ensure news_url column is clean (no NaNs)
# This step is crucial before attempting to scrape
initial_rows = len(gossip_real)
gossip_real = gossip_real.dropna(subset=['news_url']).reset_index(drop=True)
if len(gossip_real) < initial_rows:
    print(f"Dropped {initial_rows - len(gossip_real)} rows with missing 'news_url'.")


# --- Function to scrape article text from a single URL ---
def scrape_article_text(url):
    """
    Fetches the content of a URL and extracts the main article text.
    Automatically prepends 'https://' if the URL has no scheme.

    Args:
        url (str): The URL of the web page to scrape.

    Returns:
        str: The extracted article text, or an informative error message if scraping fails.
    """
    # Standardize URL: Add 'https://' if no scheme is present
    parsed_url = urlparse(url)
    if not parsed_url.scheme:
        # Default to https, as most modern sites use it
        standardized_url = f"https://{url}"
    else:
        standardized_url = url

    try:
        # Send a GET request to the standardized URL
        response = requests.get(standardized_url, headers=HEADERS, timeout=15) # Increased timeout
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # --- Article Text Extraction Logic ---
        # This is a general approach. For more precise extraction, you might need
        # to inspect the HTML structure of the specific websites you are scraping
        # (e.g., using browser developer tools) and target specific tags/classes.
        # Common patterns include:
        # - <article> tag: soup.find('article')
        # - div with specific class: soup.find('div', class_='article-body')
        # - div with specific ID: soup.find('div', id='main-content')
        # For now, we'll collect text from all <p> tags, which is often effective.

        paragraphs = soup.find_all('p')
        text_content = ' '.join([p.get_text() for p in paragraphs])

        # Basic text cleaning: remove extra whitespace and newlines
        text_content = re.sub(r'\s+', ' ', text_content).strip()

        if not text_content:
            # If no paragraph text is found, try to get text from the body or a common article div
            body_text = soup.body.get_text(separator=' ', strip=True) if soup.body else ""
            if body_text and len(body_text) > 100: # Check if body text is substantial
                text_content = re.sub(r'\s+', ' ', body_text).strip()
            else:
                return f"[No substantial text found or page structure is unusual for {standardized_url}]"

        return text_content

    except requests.exceptions.MissingSchema:
        return f"[Error fetching URL: Invalid URL '{url}': Missing scheme (e.g., http:// or https://). This error should ideally be caught by standardization logic.]"
    except requests.exceptions.ConnectionError as e:
        return f"[Error fetching URL: Connection error for {standardized_url}: {e}]"
    except requests.exceptions.Timeout:
        return f"[Error fetching URL: Request timed out for {standardized_url}]"
    except requests.exceptions.HTTPError as e:
        return f"[Error fetching URL: HTTP error {e.response.status_code} for {standardized_url}]"
    except requests.exceptions.RequestException as e:
        return f"[Error fetching URL: General request error for {standardized_url}: {e}]"
    except Exception as e:
        return f"[An unexpected error occurred for {standardized_url}: {e}]"

# --- Apply the scraping function to the 'news_url' column for the gossip_real DataFrame ---
print("\nStarting article text extraction from URLs for 'gossip_real' DataFrame. This may take some time...")
print(f"Processing {len(gossip_real)} entries.")

# Create a new column to store the extracted text
gossip_real['extracted_article_text'] = None

for index, row in gossip_real.iterrows():
    url = row['news_url']
    if pd.notna(url): # Ensure URL is not NaN
        print(f"Scraping {index+1}/{len(gossip_real)}: {url}")
        extracted_text = scrape_article_text(url)
        gossip_real.at[index, 'extracted_article_text'] = extracted_text
    else:
        gossip_real.at[index, 'extracted_article_text'] = "[No URL provided]"

    time.sleep(REQUEST_DELAY_SECONDS) # Pause for politeness

print("\nArticle text extraction complete for 'gossip_real' DataFrame!")

# Display the DataFrame with the new 'extracted_article_text' column
print("\n'gossip_real' DataFrame with Extracted Article Text (first 5 rows):")
print(gossip_real[['title', 'news_url', 'extracted_article_text']].head())

# You can also save the DataFrame to a CSV or other format
gossip_real.to_csv('gossip_real_with_article_text.csv', index=False)


DataFrame loaded from 'gossipcop_real.csv'.
Dropped 13 rows with missing 'news_url'.

Starting article text extraction from URLs for 'gossip_real' DataFrame. This may take some time...
Processing 16804 entries.
Scraping 1/16804: https://www.brides.com/story/teen-mom-jenelle-evans-wedding-dress
Scraping 2/16804: https://www.dailymail.co.uk/tvshowbiz/article-4668922/Kylie-Jenner-refusing-discuss-dating-life-new-show.html
Scraping 3/16804: https://en.wikipedia.org/wiki/Quinn_Perkins
Scraping 4/16804: https://www.refinery29.com/en-us/2018/03/191922/kim-kardashian-butt-workout-strength-routine
Scraping 5/16804: https://www.cnn.com/2017/10/04/entertainment/celine-dion-vegas-shooting-victims/index.html
Scraping 6/16804: https://tech2.org/chris-evans-millie-bobby-brown-snoop-dogg-and-more-stars-shower-bullied-student-keaton-jones-with-support/
Scraping 7/16804: https://tvline.com/2018/05/02/the-handmaids-tale-renewed-season-3-hulu/
Scraping 8/16804: https://fashionmagazine.com/culture/selena-g

/tmp/ipykernel_5740/2977579220.py:56: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.text, 'html.parser')


Scraping 188/16804: http://celebrityinsider.org/french-montana-jokes-about-alessandra-ambrosios-mtv-vmas-look-says-its-not-easy-access-62638/
Scraping 189/16804: https://www.nickiswift.com/114528/chrissy-metzs-tragic-real-life-story/
Scraping 190/16804: https://www.msn.com/en-us/movies/celebrity/heidi-klum-says-she%E2%80%99s-very-single-and-very-ready-to-mingle-during-ellen-show-appearance/vp-BBL6Dzr
Scraping 191/16804: http://www.bravotv.com/the-daily-dish/iggy-azalea-discusses-paris-hiltons-house
Scraping 192/16804: https://www.statista.com/statistics/266669/golden-globes--number-of-viewers/
Scraping 193/16804: https://www.tmz.com/2017/04/06/shia-labeouf-bowling-alley-confrontation-racist-drunk/
Scraping 194/16804: https://www.dailymail.co.uk/tvshowbiz/article-4290784/Reese-Witherspoon-Ava-twins-lunch.html
Scraping 195/16804: http://www.inglewoodtoday.com/amanda-stanton-tweets-about-unhappy-people-amid-robby-hayes-drama/
Scraping 196/16804: http://gossipbucket.com/e-online/1245425/50

In [ ]:
gossiped_real = pd.read_csv('gossip_real_with_article_text.csv')
gossiped_real.head()

In [ ]:
gossiped_real.info()

### Politifact_fake.csv

In [32]:
# Load Politifact fake
politifact_fake = pd.read_csv("politifact_fake.csv")
politifact_fake.head()

,id,news_url,title,tweet_ids
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...


In [33]:
# Inspect column attributes
politifact_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         432 non-null    object
 1   news_url   428 non-null    object
 2   title      432 non-null    object
 3   tweet_ids  392 non-null    object
dtypes: object(4)
memory usage: 13.6+ KB


In [34]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re # For cleaning up text
from urllib.parse import urlparse # Correctly imported now

# --- Configuration ---
# Set a User-Agent to mimic a web browser. Some sites block requests without this.
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
# Small delay between requests to avoid overwhelming servers and getting blocked.
REQUEST_DELAY_SECONDS = 0.5

# --- politifact_fake DataFrame ---
# Assuming politifact_fake is already loaded in your environment.
# For demonstration purposes, if it's not actually loaded, you might get a NameError.
# If you need a sample for testing, uncomment and use the following:
# data = {
#     'id': [f'politifact_fake_{i}' for i in range(10)],
#     'news_url': [
#         'www.example.com/fake-news-a',
#         'www.example.com/fake-news-b',
#         'www.example.com/fake-news-c',
#         'www.example.com/fake-news-d',
#         'www.example.com/fake-news-e',
#         'www.example.com/fake-news-f',
#         'www.example.com/fake-news-g',
#         'www.example.com/fake-news-h',
#         'www.example.com/fake-news-i',
#         'www.example.com/fake-news-j'
#     ],
#     'title': [f'Fake Title {i}' for i in range(10)],
#     'tweet_ids': [f'fake_tweet_{i}' for i in range(10)]
# }
# politifact_fake = pd.DataFrame(data)
# print("Sample politifact_fake DataFrame created for demonstration.")

# Ensure news_url column is clean (no NaNs)
# This step is crucial before attempting to scrape
initial_rows = len(politifact_fake)
politifact_fake = politifact_fake.dropna(subset=['news_url']).reset_index(drop=True)
if len(politifact_fake) < initial_rows:
    print(f"Dropped {initial_rows - len(politifact_fake)} rows with missing 'news_url' in politifact_fake.")


# --- Function to scrape article text from a single URL ---
def scrape_article_text(url):
    """
    Fetches the content of a URL and extracts the main article text.
    Automatically prepends 'https://' if the URL has no scheme.

    Args:
        url (str): The URL of the web page to scrape.

    Returns:
        str: The extracted article text, or an informative error message if scraping fails.
    """
    # Standardize URL: Add 'https://' if no scheme is present
    parsed_url = urlparse(url)
    if not parsed_url.scheme:
        # Default to https, as most modern sites use it
        standardized_url = f"https://{url}"
    else:
        standardized_url = url

    try:
        # Send a GET request to the standardized URL
        response = requests.get(standardized_url, headers=HEADERS, timeout=15) # Increased timeout
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # --- Article Text Extraction Logic ---
        # This is a general approach. For more precise extraction, you might need
        # to inspect the HTML structure of the specific websites you are scraping
        # (e.g., using browser developer tools) and target specific tags/classes).
        # Common patterns include:
        # - <article> tag: soup.find('article')
        # - div with specific class: soup.find('div', class_='article-body')
        # - div with specific ID: soup.find('div', id='main-content')
        # For now, we'll collect text from all <p> tags, which is often effective.

        paragraphs = soup.find_all('p')
        text_content = ' '.join([p.get_text() for p in paragraphs])

        # Basic text cleaning: remove extra whitespace and newlines
        text_content = re.sub(r'\s+', ' ', text_content).strip()

        if not text_content:
            # If no paragraph text is found, try to get text from the body or a common article div
            body_text = soup.body.get_text(separator=' ', strip=True) if soup.body else ""
            if body_text and len(body_text) > 100: # Check if body text is substantial
                text_content = re.sub(r'\s+', ' ', body_text).strip()
            else:
                return f"[No substantial text found or page structure is unusual for {standardized_url}]"

        return text_content

    except requests.exceptions.MissingSchema:
        return f"[Error fetching URL: Invalid URL '{url}': Missing scheme (e.g., http:// or https://). This error should ideally be caught by standardization logic.]"
    except requests.exceptions.ConnectionError as e:
        return f"[Error fetching URL: Connection error for {standardized_url}: {e}]"
    except requests.exceptions.Timeout:
        return f"[Error fetching URL: Request timed out for {standardized_url}]"
    except requests.exceptions.HTTPError as e:
        return f"[Error fetching URL: HTTP error {e.response.status_code} for {standardized_url}]"
    except requests.exceptions.RequestException as e:
        return f"[Error fetching URL: General request error for {standardized_url}: {e}]"
    except Exception as e:
        return f"[An unexpected error occurred for {standardized_url}: {e}]"

# --- Apply the scraping function to the 'news_url' column for the politifact_fake DataFrame ---
print("\nStarting article text extraction from URLs for 'politifact_fake' DataFrame. This may take some time...")
print(f"Processing {len(politifact_fake)} entries.")

# Create a new column to store the extracted text
politifact_fake['extracted_article_text'] = None

for index, row in politifact_fake.iterrows():
    url = row['news_url']
    if pd.notna(url): # Ensure URL is not NaN
        # print(f"Scraping {index+1}/{len(politifact_fake)}: {url}")
        extracted_text = scrape_article_text(url)
        politifact_fake.at[index, 'extracted_article_text'] = extracted_text
    else:
        politifact_fake.at[index, 'extracted_article_text'] = "[No URL provided]"

    time.sleep(REQUEST_DELAY_SECONDS) # Pause for politeness

print("\nArticle text extraction complete for 'politifact_fake' DataFrame!")

# Display the DataFrame with the new 'extracted_article_text' column
print("\n'politifact_fake' DataFrame with Extracted Article Text (first 5 rows):")
print(politifact_fake[['title', 'news_url', 'extracted_article_text']].head())

# You can also save the DataFrame to a CSV or other format
politifact_fake.to_csv('politifact_fake_with_article_text.csv', index=False)


Dropped 4 rows with missing 'news_url' in politifact_fake.

Starting article text extraction from URLs for 'politifact_fake' DataFrame. This may take some time...
Processing 428 entries.
Scraping 1/428: speedtalk.com/forum/viewtopic.php?t=51650
Scraping 2/428: politics2020.info/index.php/2018/03/13/court-orders-obama-to-pay-400-million-in-restitution/
Scraping 3/428: www.nscdscamps.org/blog/category/parenting/467344/update-second-roy-moore-accuser-works-for-michelle-obama-right-now-departed
Scraping 4/428: https://howafrica.com/oscar-pistorius-attempts-commit-suicide/
Scraping 5/428: http://washingtonsources.org/trump-votes-for-death-penalty-for-being-gay-5/
Scraping 6/428: gloria.tv/video/yRrtUtTCfPga6cq2VDJPcgQe4
Scraping 7/428: http://blogs.trendolizer.com/2015/01/new-york-man-wanted-for-infecting-240-men-and-women-with-hiv.html
Scraping 8/428: https://web.archive.org/web/20171027105356/http://www.religionmind.com:80/2017/10/saudi-arabia-behead-6-school-girls-for.html
Scraping 9/428

In [35]:
political_fake = pd.read_csv('politifact_fake_with_article_text.csv')
political_fake.head()

,id,news_url,title,tweet_ids,extracted_article_text
0,politifact15014,speedtalk.com/forum/viewtopic.php?t=51650,BREAKING: First NFL Team Declares Bankruptcy O...,937349434668498944\t937379378006282240\t937380...,[Error fetching URL: HTTP error 404 for https:...
1,politifact15156,politics2020.info/index.php/2018/03/13/court-o...,Court Orders Obama To Pay $400 Million In Rest...,972666281441878016\t972678396575559680\t972827...,[Error fetching URL: Connection error for http...
2,politifact14745,www.nscdscamps.org/blog/category/parenting/467...,UPDATE: Second Roy Moore Accuser Works For Mic...,929405740732870656\t929439450400264192\t929439...,Pkv Games adalah salah satu provider judi onli...
3,politifact14355,https://howafrica.com/oscar-pistorius-attempts...,Oscar Pistorius Attempts To Commit Suicide,886941526458347521\t887011300278194176\t887023...,[Error fetching URL: HTTP error 404 for https:...
4,politifact15371,http://washingtonsources.org/trump-votes-for-d...,Trump Votes For Death Penalty For Being Gay,915205698212040704\t915242076681506816\t915249...,Ready or Not Los Suenos Stories RUNE Free Down...


In [36]:
political_fake.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428 entries, 0 to 427
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      428 non-null    object
 1   news_url                428 non-null    object
 2   title                   428 non-null    object
 3   tweet_ids               389 non-null    object
 4   extracted_article_text  428 non-null    object
dtypes: object(5)
memory usage: 16.8+ KB


### politifact_real.csv

In [27]:
# Load Politifact real
politifact_real = pd.read_csv("politifact_real.csv")
politifact_real.head()

,id,news_url,title,tweet_ids
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele...",NaN
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",89804710374154240\t91270460595109888\t96039619...


In [28]:
# Inspect column attributes
politifact_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         624 non-null    object
 1   news_url   567 non-null    object
 2   title      624 non-null    object
 3   tweet_ids  409 non-null    object
dtypes: object(4)
memory usage: 19.6+ KB


In [29]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re # For cleaning up text
from urllib.parse import urlparse # Correctly imported now

# --- Configuration ---
# Set a User-Agent to mimic a web browser. Some sites block requests without this.
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
# Small delay between requests to avoid overwhelming servers and getting blocked.
REQUEST_DELAY_SECONDS = 0.5

# --- politifact_real DataFrame ---
# Assuming politifact_real is already loaded in your environment.
# If it's not actually loaded, you might get a NameError.
# No sample DataFrame creation needed as per your confirmation.


# Ensure news_url column is clean (no NaNs)
# This step is crucial before attempting to scrape
initial_rows = len(politifact_real)
politifact_real = politifact_real.dropna(subset=['news_url']).reset_index(drop=True)
if len(politifact_real) < initial_rows:
    print(f"Dropped {initial_rows - len(politifact_real)} rows with missing 'news_url' in politifact_real.")


# --- Function to scrape article text from a single URL ---
def scrape_article_text(url):
    """
    Fetches the content of a URL and extracts the main article text.
    Automatically prepends 'https://' if the URL has no scheme.

    Args:
        url (str): The URL of the web page to scrape.

    Returns:
        str: The extracted article text, or an informative error message if scraping fails.
    """
    # Standardize URL: Add 'https://' if no scheme is present
    parsed_url = urlparse(url)
    if not parsed_url.scheme:
        # Default to https, as most modern sites use it
        standardized_url = f"https://{url}"
    else:
        standardized_url = url

    try:
        # Send a GET request to the standardized URL
        response = requests.get(standardized_url, headers=HEADERS, timeout=15) # Increased timeout
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')

        # --- Article Text Extraction Logic ---
        # This is a general approach. For more precise extraction, you might need
        # to inspect the HTML structure of the specific websites you are scraping
        # (e.g., using browser developer tools) and target specific tags/classes).
        # Common patterns include:
        # - <article> tag: soup.find('article')
        # - div with specific class: soup.find('div', class_='article-body')
        # - div with specific ID: soup.find('div', id='main-content')
        # For now, we'll collect text from all <p> tags, which is often effective.

        paragraphs = soup.find_all('p')
        text_content = ' '.join([p.get_text() for p in paragraphs])

        # Basic text cleaning: remove extra whitespace and newlines
        text_content = re.sub(r'\s+', ' ', text_content).strip()

        if not text_content:
            # If no paragraph text is found, try to get text from the body or a common article div
            body_text = soup.body.get_text(separator=' ', strip=True) if soup.body else ""
            if body_text and len(body_text) > 100: # Check if body text is substantial
                text_content = re.sub(r'\s+', ' ', body_text).strip()
            else:
                return f"[No substantial text found or page structure is unusual for {standardized_url}]"

        return text_content

    except requests.exceptions.MissingSchema:
        return f"[Error fetching URL: Invalid URL '{url}': Missing scheme (e.g., http:// or https://). This error should ideally be caught by standardization logic.]"
    except requests.exceptions.ConnectionError as e:
        return f"[Error fetching URL: Connection error for {standardized_url}: {e}]"
    except requests.exceptions.Timeout:
        return f"[Error fetching URL: Request timed out for {standardized_url}]"
    except requests.exceptions.HTTPError as e:
        return f"[Error fetching URL: HTTP error {e.response.status_code} for {standardized_url}]"
    except requests.exceptions.RequestException as e:
        return f"[Error fetching URL: General request error for {standardized_url}: {e}]"
    except Exception as e:
        return f"[An unexpected error occurred for {standardized_url}: {e}]"

# --- Apply the scraping function to the 'news_url' column for the politifact_real DataFrame ---
print("\nStarting article text extraction from URLs for 'politifact_real' DataFrame. This may take some time...")
print(f"Processing {len(politifact_real)} entries.")

# Create a new column to store the extracted text
politifact_real['extracted_article_text'] = None

for index, row in politifact_real.iterrows():
    url = row['news_url']
    if pd.notna(url): # Ensure URL is not NaN
        # print(f"Scraping {index+1}/{len(politifact_real)}: {url}")
        extracted_text = scrape_article_text(url)
        politifact_real.at[index, 'extracted_article_text'] = extracted_text
    else:
        politifact_real.at[index, 'extracted_article_text'] = "[No URL provided]"

    time.sleep(REQUEST_DELAY_SECONDS) # Pause for politeness

print("\nArticle text extraction complete for 'politifact_real' DataFrame!")

# Display the DataFrame with the new 'extracted_article_text' column
print("\n'politifact_real' DataFrame with Extracted Article Text (first 5 rows):")
print(politifact_real[['title', 'news_url', 'extracted_article_text']].head())

# You can also save the DataFrame to a CSV or other format
politifact_real.to_csv('politifact_real_with_article_text.csv', index=False)


Dropped 57 rows with missing 'news_url' in politifact_real.

Starting article text extraction from URLs for 'politifact_real' DataFrame. This may take some time...
Processing 567 entries.
Scraping 1/567: http://www.nfib-sbet.org/
Scraping 2/567: http://www.cq.com/doc/newsmakertranscripts-4942265?3
Scraping 3/567: https://web.archive.org/web/20080204072132/http://www.rockymountainnews.com:80/news/2008/feb/02/romney-tells-denver-crowd-he-can-salvage-economy/
Scraping 4/567: https://web.archive.org/web/20110811143753/http://www.dems.gov/press/democratic-leaders-say-house-democrats-are-united-against-gop-default-act
Scraping 5/567: https://web.archive.org/web/20070820164107/http://www.whitehouse.gov/omb/budget/fy2008/outlook.html
Scraping 6/567: http://www.politifact.com/truth-o-meter/statements/2016/jan/21/donald-trump/donald-trump-exaggerates-when-he-says-china-has-to/
Scraping 7/567: https://www.law.cornell.edu/constitution/amendmentxxv
Scraping 8/567: http://americaneedsmitt.com/blog/2

/tmp/ipykernel_5740/2410098743.py:56: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.text, 'html.parser')


Scraping 475/567: http://www.pewforum.org/2008/09/19/pastors-to-protest-irs-rules-on-political-advocacy/
Scraping 476/567: http://wfpl.org/post/mitch-mcconnell-alison-lundergan-grimes-have-sharp-differences-campaign-finance-rules
Scraping 477/567: https://web.archive.org/web/20090113114854/http://thepage.time.com:80/obamas-full-remarks-at-chicago-credit-cards-event/
Scraping 478/567: http://www.politifact.com/truth-o-meter/statements/2012/aug/15/stephanie-cutter/ryans-plan-includes-700-billion-medicare-cuts-says/
Scraping 479/567: http://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=110&session=1&vote=00126
Scraping 480/567: http://twitter.com/JimDeMint/status/10288400197
Scraping 481/567: http://www.fedupthebook.com/
Scraping 482/567: http://www.nytimes.com/2009/09/10/us/politics/10obama.text.html
Scraping 483/567: https://web.archive.org/web/20071020151813/http://capitaleye.org:80/inside.asp?ID=311
Scraping 484/567: http://politifact.com/punditfact/st

In [30]:
political_real = pd.read_csv('politifact_real_with_article_text.csv')
political_real.head()

,id,news_url,title,tweet_ids,extracted_article_text
0,politifact14984,http://www.nfib-sbet.org/,National Federation of Independent Business,967132259869487105\t967164368768196609\t967215...,[No substantial text found or page structure i...
1,politifact12944,http://www.cq.com/doc/newsmakertranscripts-494...,comments in Fayetteville NC,942953459\t8980098198\t16253717352\t1668513250...,CQ Username Password Stay signed in Forgot you...
2,politifact333,https://web.archive.org/web/20080204072132/htt...,"Romney makes pitch, hoping to close deal : Ele...",NaN,Photo by Matt McClain Governor Mitt Romney stu...
3,politifact4358,https://web.archive.org/web/20110811143753/htt...,Democratic Leaders Say House Democrats Are Uni...,NaN,Transcript of Press Availability After House D...
4,politifact779,https://web.archive.org/web/20070820164107/htt...,"Budget of the United States Government, FY 2008",89804710374154240\t91270460595109888\t96039619...,The President’s 2008 Budget addresses the Nati...


In [31]:
political_real.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567 entries, 0 to 566
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      567 non-null    object
 1   news_url                567 non-null    object
 2   title                   567 non-null    object
 3   tweet_ids               373 non-null    object
 4   extracted_article_text  567 non-null    object
dtypes: object(5)
memory usage: 22.3+ KB
